In [1]:
early = "https://memory-alpha.fandom.com/wiki/Starfleet_casualties_(22nd_century)"
middle = "https://memory-alpha.fandom.com/wiki/Starfleet_casualties_(23rd_century)"
late = "https://memory-alpha.fandom.com/wiki/Starfleet_casualties_(24th_century)"

all_pages = [early, middle, late]

In [2]:
from bs4 import BeautifulSoup
import requests, time

In [3]:
html = []
for i in all_pages:
    r = requests.get(i)
    html.append(r.text)
    time.sleep(2)

In [4]:
soups = []
for h in html: 
    soup = BeautifulSoup(h)
    soups.append(soup)

In [5]:
rows = []
for bs in soups:
    tables = bs.find_all("table", {"class": "grey chart"})
    for i in tables:
        heads = " ".join([j.text.lower() for j in i.find_all("th")])
        if "revival" not in heads:
            these_rows = i.find_all("tr")
            for row in these_rows:
                if len(row.find_all("td")) > 1:
                    rows.append(row)

In [6]:
len(rows)

226

In [7]:
results = []
year = []
for row in rows:
    # date uses rowspan to imply repetition
    cols = row.find_all("td")
    rowlen = len(cols)
    if rowlen == 8:
        year.append(cols[0])
        cols = cols[1:]
        rowlen = len(cols)
    if rowlen == 7:
        # year is separate
        # rank, image, name, species, position, cause, reference
        try: 
            color = cols[0]['style'].split(":")[1].strip()
        except:
            color = "Unknown/Other"
        try: 
            rank = cols[0].find_all("area")[0]['title'].strip()
        except:
            rank = "Other/No Rank"
        try: 
            designation = cols[1].find_all("area")[0]['title'].strip()
        except:
            designation = "Other/No Designation"
        results.append([year[-1].text.strip(), color, cols[2].text.strip(), cols[3].text.strip(), cols[4].text.strip(), rank, designation, cols[5].text.strip(), cols[6].text.strip()])
        

In [8]:
colors = [i[1] for i in results]

In [9]:
from collections import Counter 
Counter(colors).most_common()

[('#AD722C', 59),
 ('#A01502', 24),
 ('transparent', 21),
 ('#5B1414', 18),
 ('#8C212B', 15),
 ('#143F48', 11),
 ('Unknown/Other', 7),
 ('#62749A', 7),
 ('#B59C5A', 6),
 ('#FEFEFE', 5),
 ('#742222', 4),
 ('#B87333', 4),
 ('#C0C0C0', 4),
 ('#F79C07;', 4),
 ('#0A0000', 4),
 ('grey', 4),
 ('#F6DBD0', 3),
 ('#98925E', 3),
 ('#B59C5A;', 3),
 ('#FFFFFF', 2),
 ('goldenrod;', 2),
 ('#FEFEFE;', 2),
 ('gold', 2),
 ('#112959', 2),
 ('#073835', 1),
 ('#1A3F73', 1),
 ('#FFFC00', 1),
 ('#FE0100', 1),
 ('#CC0000', 1),
 ('#B87333;', 1),
 ('#62749A;', 1),
 ('#A7A0AB', 1),
 ('blank', 1),
 ('black', 1)]

In [10]:
color_groups = {'#AD722C': 'Yellow', '#A01502':'Red', '#5B1414':'Red', '#8C212B':'Red', '#143F48':'Green/Blue',
          '#62749A':'Green/Blue', '#B59C5A':'Yellow', '#FEFEFE':'White', 'transparent':'Unknown/Other', 
          'Unknown/Other':'Unknown/Other', '#742222':'Red', '#F79C07;':'Yellow', '#0A0000':'Black', 
          'grey':'Gray', '#B87333':'Yellow', '#F6DBD0':'Yellow', '#98925E':'Yellow', '#B59C5A;':'Yellow',
          '#FFFFFF':'White', '#C0C0C0':'Gray', 'goldenrod;':'Yellow', '#FEFEFE;':'White', 'gold':'Yellow',
          '#112959':'Green/Blue', '#073835': 'Green/Blue', '#1A3F73':'Green/Blue', '#FFFC00':'Yellow', '#FE0100':'Red',
          '#CC0000':'Red', '#B87333;':'Yellow', '62749A;':'Green/Blue', '#A7A0AB':'Gray', 'blank':'Unknown/Other',
          'black':'Black', '#62749A;':'Green/Blue',}

In [11]:
groups = []
for i in colors:
    try:
        group = color_groups[i]
        groups.append(group)
    except:
        print(i)

In [12]:
Counter(groups).most_common()

[('Yellow', 88),
 ('Red', 63),
 ('Unknown/Other', 29),
 ('Green/Blue', 23),
 ('White', 9),
 ('Gray', 9),
 ('Black', 5)]

In [13]:
import pandas as pd
cols = ['stardate', 'color', 'name', 'species', 'position', 'rank', 'designation', 'description', 'source']
df = pd.DataFrame.from_records(results, columns=cols)
df['color_group'] = groups
df

,stardate,color,name,species,position,rank,designation,description,source,color_group
0,2153,#5B1414,Fuller,Human,Tactical officer,Crewman,Other/No Designation,Killed by Osaarian pirates,"ENT: ""Anomaly""",Red
1,2153,Unknown/Other,Sim,Mimetic simbiot,,Other/No Rank,Other/No Designation,Created to provide Charles Tucker III with vit...,"ENT: ""Similitude""",Unknown/Other
2,2153,#5B1414,Unnamed,Human,,Crewman,Other/No Designation,Shot by a Xindi bio weapon,"ENT: ""Rajiin""",Red
3,2153,#5B1414,Unnamed,Human,,Crewman,Other/No Designation,Shot by a Xindi bio weapon,"ENT: ""Rajiin""",Red
4,2153,#073835,Unnamed,Human,,Crewman,Other/No Designation,Killed by a Triannon organic explosive and the...,"ENT: ""Chosen Realm""",Green/Blue
5,2153 | 2154,transparent,"O'Malley, Patricia F.",Human,,Ensign,Other/No Designation,"Died ""in a battle with a hostile species""","ENT: ""Storm Front, Part II""",Unknown/Other
6,2154,#5B1414,Burrows,Human,Armory officer,Ensign,Other/No Designation,Killed by the Quinn Erickson subspace entity,"ENT: ""Daedalus""",Red
7,2154,transparent,Kamata,Human,,Crewman,Other/No Designation,Trapped in section decompression during a Xind...,"ENT: ""The Forgotten""",Unknown/Other
8,2154,transparent,Marcel,Human,,Ensign,Other/No Designation,Killed during a Xindi attack,"ENT: ""The Forgotten""",Unknown/Other
9,2154,#5B1414,"Taylor, Jane",Human,EPS control specialist,Lieutenant,Other/No Designation,Trapped in section decompression during a Xind...,"ENT: ""The Forgotten""",Red


In [109]:
df.to_csv("uncorrected_death_scrape.csv")